In [10]:
import io
import pandas as pd
import requests
import pyarrow #Required to read parquet files

In [14]:
df = pd.read_parquet('data/yellow_tripdata_2024-01.parquet', engine= 'pyarrow')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN


In [21]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,1
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,2
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,3
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,4


In [25]:
dim_datetime = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
dim_datetime['pick_hour'] = df['tpep_pickup_datetime'].dt.hour
dim_datetime['pick_day'] = df['tpep_pickup_datetime'].dt.day
dim_datetime['pick_weekday'] = df['tpep_pickup_datetime'].dt.weekday
dim_datetime['pick_month'] = df['tpep_pickup_datetime'].dt.month
dim_datetime['pick_year'] = df['tpep_pickup_datetime'].dt.year

dim_datetime['drop_hour'] = df['tpep_dropoff_datetime'].dt.hour
dim_datetime['drop_day'] = df['tpep_dropoff_datetime'].dt.day
dim_datetime['drop_weekday'] = df['tpep_dropoff_datetime'].dt.weekday
dim_datetime['drop_month'] = df['tpep_dropoff_datetime'].dt.month
dim_datetime['drop_year'] = df['tpep_dropoff_datetime'].dt.year

dim_datetime.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_weekday,pick_month,pick_year,drop_hour,drop_day,drop_weekday,drop_month,drop_year
0,2024-01-01 00:57:55,2024-01-01 01:17:43,0,1,0,1,2024,1,1,0,1,2024
1,2024-01-01 00:03:00,2024-01-01 00:09:36,0,1,0,1,2024,0,1,0,1,2024
2,2024-01-01 00:17:06,2024-01-01 00:35:01,0,1,0,1,2024,0,1,0,1,2024
3,2024-01-01 00:36:38,2024-01-01 00:44:56,0,1,0,1,2024,0,1,0,1,2024
4,2024-01-01 00:46:51,2024-01-01 00:52:57,0,1,0,1,2024,0,1,0,1,2024
